# Extracción y procesamiento de datos multimodales en vídeos TED

Este notebook implementa el pipeline completo descrito en el apartado **3.2 del TFM**, correspondiente al **procesamiento de vídeos y construcción del dataset multimodal**.

Incluye tareas de descarga, transcripción automática, segmentación dinámica, extracción de características acústicas y visuales, y organización de los resultados por vídeo y por lote.

Los datos procesados se almacenan en formato estructurado (JSON), y posteriormente integrados en estructuras tabulares para su análisis.


⚠️ Requisitos importantes antes de ejecutar este notebook
Este notebook está diseñado para ejecutarse en Google Colab.
Requiere el uso de una GPU T4 y una correcta configuración de rutas para funcionar correctamente.

1. Tipo de entorno
Ve a Entorno de ejecución > Cambiar tipo de entorno y selecciona GPU T4
Comprueba que se ha asignado una GPU T4 ejecutando:
2. Instalación de librerías
Tras la instalación de las librerías necesarias, es obligatorio reiniciar el entorno antes de continuar con la ejecución.

Ve a Entorno de ejecución > Reiniciar entorno de ejecución

3. Configuración de rutas
Antes de ejecutar las celdas principales, configura correctamente las siguientes rutas:

folder_path: Carpeta de trabajo principal.

json_path: Carpeta donde descargaremos los json, a meddida que se vayan analizando los videos.

model_path: Carpeta donde está guardado el modelo de emociones audio


In [3]:
!pip install faster-whisper librosa torchaudio ultralytics yt-dlp opencv-python pandas matplotlib mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 6.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s

# **IMPORTACIONES**

In [1]:
import os
import warnings

# ---- Ignorar warnings de Python ----
warnings.filterwarnings("ignore")


import torch
import torchaudio
import librosa
import joblib
import pandas as pd
from ultralytics import YOLO
import mediapipe as mp


import time
import json
import numpy as np
import subprocess
import tempfile
from faster_whisper import WhisperModel
import yt_dlp

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import math
import random
import cv2
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Configuración del entorno y rutas de trabajo

Se definen las rutas de entrada y salida donde se almacenarán los vídeos descargados, las transcripciones, las características extraídas y los resultados finales.

También se configuran los parámetros globales del procesamiento de audio (frecuencia de muestreo, tamaño de ventana, etc.).


In [2]:
from google.colab import drive
drive.mount('/content/drive')
#carpeta de trabajo principal
folder_path = "/content/drive/MyDrive/Analisis_Multimodal_Comunicacion_TFM/data/folder_path"
#carpeta para guardar los archivos conforme se vallan analizando videos
json_path = "/content/drive/MyDrive/Analisis_Multimodal_Comunicacion_TFM/data/json_path"
#carpeta donde está el modelo de emodiones del audio
model_path = "/content/drive/MyDrive/Analisis_Multimodal_Comunicacion_TFM/models"



os.makedirs(folder_path, exist_ok=True)
os.makedirs(json_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Configuración del dispositivo y carga del modelo Whisper

Se detecta automáticamente si hay una GPU disponible para acelerar el procesamiento. Luego se inicializa el modelo Whisper (`small`) para realizar la transcripción automática de los vídeos, con soporte para múltiples idiomas.

Este modelo se usará más adelante para obtener la transcripción cronometrada de cada vídeo, paso fundamental para la segmentación dinámica y análisis textual.


In [ ]:
# =====================
# Config
# =====================
SAMPLE_RATE = 16000
FRAME_LENGTH = 2048
HOP_LENGTH = 512
EMPHASIS_LEVELS = 10

# =====================
# Definir device
# =====================

# Configuración inicial
device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)

# Verificación detallada del dispositivo
print(f"\n{'='*50}")
print(f"Configuración de Dispositivo:")
print(f"Tipo: {device_type.upper()}")
if device_type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Capacidad: {torch.cuda.get_device_capability()}")
    print(f"Memoria Total: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f} GB")
print(f"{'='*50}\n")

# Usar float32 por defecto para mayor estabilidad
torch.set_default_dtype(torch.float32)
compute_type = "float32"  # Para Whisper

# =========================================
# Definir modelo para extracción del texto
# =========================================

model_whisper = WhisperModel(
    "small",
    device=device_type,
    compute_type=compute_type,
)
print(f"Modelo Whisper configurado en {device_type} con compute_type={compute_type}")


def clean_up():
    if device_type == "cuda":
        torch.cuda.empty_cache()



Configuración de Dispositivo:
Tipo: CUDA
GPU: NVIDIA A100-SXM4-80GB
Capacidad: (8, 0)
Memoria Total: 79.32 GB



tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

Modelo Whisper configurado en cuda con compute_type=float32


  ## Modelo para la detección de emociones acústicas

En este bloque se carga y aplica el modelo entrenado para predecir la emoción predominante en un segmento de audio, como se describe en el apartado **3.2.5 del TFM**.

### Arquitectura del modelo

Se define una red neuronal profunda (`DeepModel`) implementada con PyTorch. Su entrada es un vector de 143 características acústicas, y su salida corresponde a una de las 8 emociones posibles (según el dataset RAVDESS). La arquitectura incluye:

- Capas totalmente conectadas (`Linear`) con activación ReLU.
- Regularización mediante `Dropout` en varias capas ocultas.
- Una capa de salida con tamaño igual al número de clases emocionales.

El modelo se carga desde disco (`deep_model.pth`) y se pasa a modo evaluación (`eval()`), utilizando `float32` para mantener la estabilidad numérica.

### Normalización y codificación

Se cargan también dos objetos entrenados previamente:
- Un **`StandardScaler`** (`scaler.pkl`) para normalizar las características antes de la predicción.
- Un **`LabelEncoder`** (`label_encoder.pkl`) para decodificar las predicciones numéricas en etiquetas emocionales (por ejemplo, *alegría*, *tristeza*, etc.).

### Funciones auxiliares

Se definen dos funciones clave:

- `pad_audio_smart()`: ajusta cualquier segmento de audio a una longitud fija (2.5 segundos) mediante recorte o padding (con ceros o parte del audio anterior), para asegurar una entrada consistente al modelo.

- `extract_features_emotion()`: calcula y concatena las características acústicas que alimentan al modelo:
  - ZCR (Zero Crossing Rate)
  - MFCC (13 coeficientes)
  - RMS (energía)
  - Mel Spectrogram (resumen por bandas)

- `predict_emotion()`: transforma el audio en características, las normaliza, las pasa por el modelo y devuelve la **emoción estimada como etiqueta** (texto).

Esta función se aplicará a cada segmento de audio extraído del corpus TED, generando el valor `emocion_audio` que se

In [ ]:
# =========================================
# Modelo para extracción sentimiento audio
# =========================================


class DeepModel(nn.Module):
    def __init__(self, input_dim=143, output_dim=8):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 64),
            nn.ReLU(),

            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# =========================================
# rutas al modelo
# =========================================
modelo_emo= os.path.join(model_path, "deep_model.pth")
Scaler= os.path.join(model_path, "scaler.pkl")
encoder= os.path.join(model_path, "label_encoder.pkl")

model_e = DeepModel(input_dim=143, output_dim=8)
model_e.load_state_dict(torch.load(modelo_emo))
model_e = model_e.to(device).to(torch.float32).eval()  # Forzar float32

scaler = joblib.load(Scaler)
le = joblib.load(encoder)

# =====================
# Funciones
# =====================
def pad_audio_smart(data, sr, target_sec=2.5, prev_data=None):

    target_len = int(target_sec * sr)

    if len(data) < target_len:
        pad_len = target_len - len(data)
        if prev_data is not None and len(prev_data) >= pad_len:
            # Tomar del final del audio anterior
            pad = prev_data[-pad_len:]
        else:
            # Rellenar con ceros
            pad = np.zeros(pad_len, dtype=data.dtype)
        data = np.concatenate([pad, data])
    else:
        # Recortar al final
        data = data[-target_len:]

    return data


def extract_features_emotion(data, sample_rate):
    # Calcular características una sola vez
    rms = np.mean(librosa.feature.rms(y=data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0], dtype=np.float32)
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0], dtype=np.float32)
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13), axis=1, dtype=np.float32)
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH), axis=1, dtype=np.float32)

    # Crear vector final
    features = np.concatenate([[zcr], mfcc, [rms], mel])
    return features

def predict_emotion(data, sr):
    features = extract_features_emotion(data, sr)
    x_features = np.array(features).reshape(1, -1)
    scaled = scaler.transform(x_features)
    with torch.no_grad():
      sample = torch.tensor(scaled, dtype=torch.float32).to(device)
      output = model_e(sample)
      pred = output.argmax(dim=1)
      predicted_label = le.inverse_transform([pred.item()])[0]
      return predicted_label



## Descarga del vídeo TED y extracción del audio

En este bloque se definen dos funciones fundamentales para el procesamiento de los vídeos TED:

### `download_video(url)`
Esta función permite descargar el vídeo original desde su URL (TED o YouTube) utilizando la herramienta `yt-dlp`. La descarga se realiza con los siguientes ajustes:

- Se selecciona la mejor combinación disponible de video y audio.
- El archivo resultante se guarda con el identificador único del vídeo como nombre (`%(id)s.mp4`).
- El vídeo se guarda en formato `.mp4`, y se suprimen tanto la salida detallada como las advertencias para simplificar el flujo en notebooks.

La función devuelve la ruta del archivo descargado y el `video_id` correspondiente.

### `extract_audio_from_video(video_path, audio_path)`
Una vez descargado el vídeo, esta función extrae únicamente la **pista de audio**, utilizando `ffmpeg`. El audio se guarda en formato **WAV sin comprimir**, con los siguientes parámetros:

- Canal único (mono): `-ac 1`
- Frecuencia de muestreo: definida por `SAMPLE_RATE`
- Codificación PCM lineal: `pcm_s16le`

Esta función es necesaria para preparar el audio en condiciones óptimas para la posterior transcripción y análisis acústico.

In [ ]:

def download_video(url):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        'merge_output_format': 'mp4',
        'quiet': True,          # Evita la mayoría de la salida
        'no_warnings': True,    # Oculta advertencias
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        video_path = f"{info['id']}.mp4"
        return video_path, info['id']

def extract_audio_from_video(video_path, audio_path):
    command = [
        'ffmpeg', '-i',video_path, '-vn',
        '-acodec', 'pcm_s16le', '-ar', str(SAMPLE_RATE),
        '-ac', '1', audio_path, '-y'
    ]
    subprocess.run(command, stdout=subprocess.DEVNULL,
                   stderr=subprocess.DEVNULL)

## Segmentación del discurso y análisis por fragmentos

Este bloque define el procedimiento principal para segmentar el audio del vídeo en fragmentos significativos y enriquecerlos con información textual y emocional. Está compuesto por tres funciones clave:

### `segmentos_por_pausa_enfasis()`

Divide el audio en segmentos dinámicos en función de dos señales acústicas:
- **RMS**: energía del audio.
- **ZCR**: tasa de cruce por cero (indicador de actividad sonora).

Los segmentos se generan si se detecta una **pausa prolongada** (`min_pause`) o un **cambio súbito de énfasis** (basado en la variación conjunta de RMS y ZCR). Se asegura una **duración mínima** (`min_seg`) para evitar fragmentos irrelevantes.

Cada segmento resultante incluye:
- Tiempos de inicio y fin.
- Energía media (`rms_mean`), sonoridad (`zcr_mean`).
- Duración de la pausa anterior (`prev_pause`).

---

### `transcribir_con_segmentos(model, y, sr, segmentos)`

Dada una señal de audio (`y`) ya segmentada:

1. Se guarda temporalmente como archivo `.wav`.
2. Se transcribe usando Whisper, con marcas de tiempo por palabra.
3. Se asocian las palabras a cada segmento en función de sus tiempos de aparición.
4. Se calcula:
   - El **texto parcial** correspondiente a cada segmento.
   - El número de **palabras por minuto** (`pmm`).
   - El **tipo** de segmento: `"Habla"` o `"Pausa"`.
   - La **emoción acústica** estimada (`emocion`) mediante el modelo previamente cargado.

El resultado es una lista estructurada de segmentos enriquecidos con datos acústicos, textuales y emocionales, además del **texto completo transcrito** y el **idioma detectado** por Whisper.

---

### `analizar_audio(audio_path)`

Función principal que encapsula todo el flujo anterior. A partir de la ruta de un archivo de audio:

1. Lo segmenta mediante `segmentos_por_pausa_enfasis()`.
2. Transcribe y analiza cada fragmento con `transcribir_con_segmentos()`.
3. Devuelve un diccionario con:
   - Duración total del vídeo.
   - Transcripción completa.
   - Idioma detectado.
   - Lista detallada de segmentos analizados.


In [ ]:
def segmentos_por_pausa_enfasis(
    audio_path,
    sr=SAMPLE_RATE,
    threshold=0.01,
    min_pause=0.4,
    min_seg=1.0,
    min_cambio_enfasis=2
):
    import librosa
    import numpy as np

    y, _ = librosa.load(audio_path, sr=sr)

    rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0]
    zcr = librosa.feature.zero_crossing_rate(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0]

    # Normalización para énfasis
    rms_norm = rms / (np.max(rms) + 1e-8)
    zcr_norm = zcr / (np.max(zcr) + 1e-8)
    combined = 0.7 * rms_norm + 0.3 * zcr_norm
    combined_norm = combined / (np.max(combined) + 1e-8)
    enfasis_levels = np.clip(np.ceil(combined_norm * EMPHASIS_LEVELS), 1, EMPHASIS_LEVELS).astype(int).tolist()

    times = librosa.frames_to_time(np.arange(len(rms)), sr=sr, hop_length=HOP_LENGTH)

    segmentos = []
    start_time = times[0]
    pause_time = 0.0
    prev_enfasis = enfasis_levels[0]
    prev_pause = 0.0

    for i in range(1, len(rms)):
        time = times[i]
        is_pause = rms[i] < threshold
        enfasis_actual = enfasis_levels[i]
        cambio_enfasis = abs(enfasis_actual - prev_enfasis) >= min_cambio_enfasis

        if is_pause:
            pause_time += times[i] - times[i - 1]
        else:
            if pause_time >= min_pause:
                end_time = time
                if end_time - start_time >= min_seg:
                    start_sample = int(start_time * sr)
                    end_sample = int(end_time * sr)

                    seg_rms = rms[(times >= start_time) & (times <= end_time)]
                    seg_zcr = zcr[(times >= start_time) & (times <= end_time)]
                    seg_wave = y[start_sample:end_sample]

                    segmentos.append({
                        "inicio": round(start_time, 2),
                        "fin": round(end_time, 2),
                        "rms_mean": float(np.mean(seg_rms)),
                        "zcr_mean": float(np.mean(seg_zcr)),
                        "prev_pause": prev_pause
                    })
                start_time = end_time
                prev_pause = pause_time
                pause_time = 0.0
            else:
                if cambio_enfasis and (time - start_time >= min_seg):
                    end_time = time
                    start_sample = int(start_time * sr)
                    end_sample = int(end_time * sr)

                    seg_rms = rms[(times >= start_time) & (times <= end_time)]
                    seg_zcr = zcr[(times >= start_time) & (times <= end_time)]
                    seg_wave = y[start_sample:end_sample]

                    segmentos.append({
                        "inicio": round(start_time, 2),
                        "fin": round(end_time, 2),
                        "rms_mean": float(np.mean(seg_rms)),
                        "zcr_mean": float(np.mean(seg_zcr)),
                        "rms_vector": seg_rms.tolist(),
                        "prev_pause": prev_pause
                    })
                    start_time = end_time
                    prev_pause = pause_time
                    pause_time = 0.0
        prev_enfasis = enfasis_actual

    # Último segmento
    if times[-1] - start_time >= min_seg:
        start_sample = int(start_time * sr)
        end_sample = len(y)

        seg_rms = rms[(times >= start_time) & (times <= times[-1])]
        seg_zcr = zcr[(times >= start_time) & (times <= times[-1])]
        seg_wave = y[start_sample:end_sample]

        segmentos.append({
            "inicio": round(start_time, 2),
            "fin": round(times[-1], 2),
            "rms_mean": float(np.mean(seg_rms)),
            "zcr_mean": float(np.mean(seg_zcr)),
            "prev_pause": prev_pause
        })

    return y, sr, segmentos

def transcribir_con_segmentos(model, y, sr, segmentos):
    segmentos_lista = []
    texto_completo = ""
    idioma_detectado = None

    # Guardar todo el audio como archivo temporal
    with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
        torchaudio.save(tmp.name, torch.tensor(y).unsqueeze(0), sample_rate=sr)

        # Transcripción con Whisper
        whisper_gen, info = model.transcribe(tmp.name, word_timestamps=True)
        idioma_detectado = getattr(info, "language", None)
        whisper_segments = list(whisper_gen)

        # Texto completo
        texto_completo = " ".join([seg.text for seg in whisper_segments])

        # Aplanar palabras
        todas_palabras = []
        for seg in whisper_segments:
            todas_palabras.extend(seg.words)

        seg_id = 0
        for seg in segmentos:
            inicio = seg["inicio"]
            fin = seg["fin"]
            rms_mean = seg["rms_mean"]
            zcr_mean = seg["zcr_mean"]
            prev_pause = seg["prev_pause"]
            seg_id += 1

            start_sample = int(inicio * sr)
            end_sample = int(fin * sr)
            corte_audio = y[start_sample:end_sample]

            # Palabras dentro del rango
            palabras_segmento = [
                w.word for w in todas_palabras
                if w.end > inicio and w.start < fin
            ]
            texto = " ".join(palabras_segmento).strip()

            if texto:
                tipo = "Habla"
                duracion_min = (fin - inicio) / 60  # sin redondear antes
                pmm = len(texto.split()) / duracion_min if duracion_min > 0 else 0
                emotion = predict_emotion(corte_audio, sr)
            else:
                tipo = "Pausa"
                pmm = 0
                emotion = ""

            segmentos_lista.append({
                "seg_id": seg_id,
                "audio": {
                    "inicio": round(inicio, 2),
                    "fin": round(fin, 2),
                    "duracion": round(fin - inicio, 2),
                    "pausa_anterior": prev_pause,
                    "rms_mean": rms_mean,
                    "zcr_mean": zcr_mean,
                    "tipo": tipo,  # cambiado a minúscula para consistencia
                    "pmm": pmm,
                    "texto": texto,
                    "emocion": emotion
                },
                "video":{
                    "t_central": round ((inicio + fin) / 2,2)
                }
            })

    return segmentos_lista, texto_completo, idioma_detectado

def analizar_audio(audio_path):
    y, sr, segmentos = segmentos_por_pausa_enfasis(audio_path)

    segmentos_lista, texto_completo,idioma = transcribir_con_segmentos(model_whisper, y, sr, segmentos)

    duracion_video = librosa.get_duration(y=y, sr=sr)

    return {
        "duracion_video": duracion_video,
        "texto_completo": texto_completo,
        "idioma": idioma,
        "segmentos": segmentos_lista
    }

## Análisis visual de los segmentos mediante YOLO y MediaPipe

Este bloque implementa el análisis visual por fotograma para cada segmento del vídeo, utilizando modelos ligeros para detectar postura, gestos faciales y movimiento de manos. Este proceso genera las variables visuales asociadas a cada fragmento, descritas en el apartado **3.2.3 del TFM**.

### 1. Inicialización de modelos

- Se carga **YOLOv8n** para detectar personas en cada fotograma.
- Se inicializan los modelos de **MediaPipe**:
  - `face_mesh`: para análisis detallado del rostro.
  - `pose`: para postura corporal.
  - `hands`: para manos y su posición.

### 2. Detección de la persona principal (`detectar_persona_principal`)

Dado un fotograma, se detectan todas las personas y se selecciona la más relevante en base a una puntuación que combina:
- **Tamaño** (área del bounding box).
- **Brillo** (luminosidad del rostro).

Este recorte centrado se analiza posteriormente con MediaPipe.

### 3. Análisis del fotograma (`analyze_frame_extended`)

Se calcula un conjunto amplio de características visuales a partir de un solo fotograma representativo de cada segmento:

#### 🧠 Cara:
- **Inclinación de la cabeza** (yaw, pitch, roll).
- **Boca abierta** (indicador de expresión o habla).
- **Sonrisa** (intensidad y detección binaria).
- **Ceño fruncido** (intensidad y detección binaria).
- **Ojos abiertos**.
- **Asimetría labial**.
- **Tensión facial** (puntuación heurística).
- **Estado emocional** inferido (e.g., *sonriente*, *tenso*, *sorprendido*, *neutral*).

#### 🧍 Postura corporal:
- **Apertura de brazos** (medida por ángulo entre articulaciones).
- **Inclinación del torso** (basada en el ángulo del tronco respecto al eje vertical).

#### ✋ Manos:
- **Detección de manos visibles**.
- **Tipo de mano** (izquierda/derecha) y **posición media**.

### 4. Procesamiento del vídeo (`procesar_video`)

Para cada segmento:
- Se calcula el **frame central** en función del tiempo.
- Se extrae dicho fotograma.
- Se aplica la detección de la persona principal.
- Se realiza el análisis visual completo.
- Los resultados se integran en el campo `"video"` del diccionario del segmento.

Este análisis se repite para todos los segmentos generados durante el procesamiento del audio.

In [ ]:
# Inicializa YOLO

yolo_model = YOLO("yolov8n.pt")

# ---- Inicialización de Mediapipe ----
def init_solutions():
    mp_face_mesh = mp.solutions.face_mesh
    mp_pose = mp.solutions.pose
    mp_hands = mp.solutions.hands
    return mp_face_mesh, mp_pose, mp_hands

# ---- Función de ángulo ----
def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# ---- Detección persona principal ----
def detectar_persona_principal(frame, yolo_model, conf_thresh=0.3, alpha=0.6):
    detections = yolo_model.predict(frame, conf=conf_thresh, verbose=False)[0]
    person_class_id = 0
    persons = [box for box in detections.boxes if int(box.cls) == person_class_id]

    if not persons:
        return frame  # no hay personas detectadas

    areas, brightness = [], []
    for box in persons:
        coords = box.xyxy.cpu().numpy().flatten()
        x1, y1, x2, y2 = map(int, coords)
        w, h = x2 - x1, y2 - y1
        areas.append(w * h)

        crop = frame[y1:y2, x1:x2]
        if crop.size == 0:
            brightness.append(0)
        else:
            gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
            brightness.append(np.mean(gray))

    areas = np.array(areas) / (np.max(areas) + 1e-6)
    brightness = np.array(brightness) / (np.max(brightness) + 1e-6)
    scores = alpha * areas + (1 - alpha) * brightness
    idx = np.argmax(scores)

    coords = persons[idx].xyxy.cpu().numpy().flatten()
    x1, y1, x2, y2 = map(int, coords)

    return frame[y1:y2, x1:x2]

# ---- Análisis de frame ----
def analyze_frame_extended(frame, frame_id, face_mesh, pose, hands):
    frame_resize = cv2.resize(frame, (320, int(frame.shape[0] * 320 / frame.shape[1])))
    rgb_frame = cv2.cvtColor(frame_resize, cv2.COLOR_BGR2RGB)
    result_dict = {
        "frame_id": frame_id,
        "cara_detectada": False,
        "inclinacion_cabeza": {"yaw": None, "pitch": None, "roll": None},
        "boca_abierta": None,
        "sonrisa": None,
        "sonrisa_detectada": False,
        "ceño_fruncido": None,
        "ceño_detectado": False,
        "ojos_abiertos": None,
        "asimetria_labios": None,
        "tension_facial": None,
        "estado_emocional": "desconocido",
        "apertura_brazos": None,
        "inclinacion_torso": None,
        "manos_visibles": False,
        "detalle_manos": []
    }

    # ---- Cara ----
    face_results = face_mesh.process(rgb_frame)
    if face_results.multi_face_landmarks:
        face = face_results.multi_face_landmarks[0]
        result_dict["cara_detectada"] = True

        # Referencias
        left_eye = face.landmark[33]
        right_eye = face.landmark[263]
        nose_tip = face.landmark[1]
        chin = face.landmark[152]

        # Distancia entre ojos para normalizar medidas
        eye_distance = np.sqrt(
            (right_eye.x - left_eye.x) ** 2 + (right_eye.y - left_eye.y) ** 2
        )

        # ---- Inclinación cabeza (yaw, pitch, roll) ----
        roll = math.degrees(math.atan2(
            right_eye.y - left_eye.y,
            right_eye.x - left_eye.x
        ))
        pitch = math.degrees(math.atan2(
            chin.y - nose_tip.y,
            chin.x - nose_tip.x
        ))
        eye_center_x = (left_eye.x + right_eye.x) / 2.0
        eye_center_y = (left_eye.y + right_eye.y) / 2.0
        yaw = math.degrees(math.atan2(
            nose_tip.x - eye_center_x,
            nose_tip.y - eye_center_y
        ))

        result_dict["inclinacion_cabeza"] = {"yaw": yaw, "pitch": pitch, "roll": roll}

        # ---- Boca, sonrisa, ceño, ojos ----
        mouth_open = abs(face.landmark[13].y - face.landmark[14].y)
        mouth_open_norm = mouth_open / (eye_distance + 1e-6)
        result_dict["boca_abierta"] = mouth_open_norm

        left_mouth, right_mouth = face.landmark[61], face.landmark[291]
        mouth_width = abs(right_mouth.x - left_mouth.x)
        mouth_height = abs(face.landmark[13].y - face.landmark[14].y)
        smile_ratio = mouth_width / (mouth_height + 1e-6)
        result_dict["sonrisa"] = smile_ratio
        result_dict["sonrisa_detectada"] = smile_ratio > 1.8

        brow_left_inner, brow_right_inner = face.landmark[70], face.landmark[300]
        brow_distance = abs(brow_right_inner.x - brow_left_inner.x) / (eye_distance + 1e-6)
        result_dict["ceño_fruncido"] = brow_distance
        result_dict["ceño_detectado"] = brow_distance < 0.04

        left_eye_open = abs(face.landmark[159].y - face.landmark[145].y)
        right_eye_open = abs(face.landmark[386].y - face.landmark[374].y)
        eye_open_avg = (left_eye_open + right_eye_open) / 2 / (eye_distance + 1e-6)
        result_dict["ojos_abiertos"] = eye_open_avg


        result_dict["asimetria_labios"] = abs(left_mouth.y - right_mouth.y)

        # ---- Tensión facial ----
        tension_score = 0
        if brow_distance < 0.04: tension_score += 1
        if eye_open_avg > 0.06: tension_score += 1
        if mouth_open_norm < 0.02: tension_score += 1
        if smile_ratio > 2.5: tension_score += 1
        result_dict["tension_facial"] = tension_score

        # ---- Estado emocional ----
        if result_dict["sonrisa_detectada"] and tension_score <= 1:
            estado = "sonriente"
        elif tension_score >= 3:
            estado = "tenso"
        elif eye_open_avg > 0.08 and mouth_open_norm > 0.08:
            estado = "sorprendido"
        else:
            estado = "neutral"
        result_dict["estado_emocional"] = estado

    # ---- Postura ----
    pose_results = pose.process(rgb_frame)
    if pose_results.pose_landmarks:
        lm = pose_results.pose_landmarks.landmark
        left_angle = calculate_angle(lm[mp.solutions.pose.PoseLandmark.LEFT_ELBOW],
                                     lm[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER],
                                     lm[mp.solutions.pose.PoseLandmark.LEFT_HIP])
        right_angle = calculate_angle(lm[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW],
                                      lm[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER],
                                      lm[mp.solutions.pose.PoseLandmark.RIGHT_HIP])
        result_dict["apertura_brazos"] = (left_angle + right_angle) / 2
        torso_angle = calculate_angle(lm[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER],
                                     lm[mp.solutions.pose.PoseLandmark.LEFT_HIP],
                                     lm[mp.solutions.pose.PoseLandmark.LEFT_KNEE])
        result_dict["inclinacion_torso"] = torso_angle

    # ---- Manos ----
    hand_results = hands.process(rgb_frame)
    if hand_results.multi_hand_landmarks:
        result_dict["manos_visibles"] = True
        hands_info = []
        for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
            label = handedness.classification[0].label
            x_list = [lm.x for lm in hand_landmarks.landmark]
            y_list = [lm.y for lm in hand_landmarks.landmark]
            hands_info.append({
                "tipo": label,
                "posicion_media": {"x": np.mean(x_list), "y": np.mean(y_list)}
            })
        result_dict["detalle_manos"] = hands_info

    return result_dict

# ---- Procesar video ----
def procesar_video(video_path, segmentos_lista):
    mp_face_mesh, mp_pose, mp_hands = init_solutions()

    with mp_face_mesh.FaceMesh(static_image_mode=False, refine_landmarks=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh, \
         mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose, \
         mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)

        for segmento in segmentos_lista:
            t_central = segmento["video"]["t_central"]
            frame_id = int(t_central * fps)

            # Mover puntero del video al frame deseado
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
            ret, frame = cap.read()
            if not ret:
                print(f"No se pudo leer el frame {frame_id}")
                continue

            # Detectar persona principal y analizar frame
            principal_frame = detectar_persona_principal(frame, yolo_model)
            resultado = analyze_frame_extended(principal_frame, frame_id, face_mesh, pose, hands)

            # Guardar resultado en el segmento
            segmento["video"]["frame_id"] = frame_id
            segmento["video"]["analisis"] = resultado

        cap.release()

    return segmentos_lista  # ← Devolvemos la lista modificada


# **RUN**

## Descarga y lectura del vídeo TED

A partir del identificador de cada vídeo (`video_id`) y su URL asociada, se descarga el archivo con `yt-dlp`. Posteriormente, se extraen por separado el audio (formato `.wav`) y los fotogramas necesarios para el análisis visual.

Este proceso se repetirá para todos los vídeos del conjunto seleccionado, organizados en grupos por lotes.

## Carga del índice general y selección de vídeos por grupo

En este bloque se carga el archivo `videos_analisis.csv`, que contiene la lista completa de vídeos seleccionados para el estudio, junto con su categoría (buen/mal comunicador) y el grupo de procesamiento al que pertenecen (0 a 99).

### `extraer_lista_urls(df, grupo)`

Esta función filtra el DataFrame por un grupo específico y devuelve una lista de tuplas con los siguientes datos por vídeo:
- Enlace al vídeo (`link`).
- Categoría (`categoria`): indica si pertenece a la clase 0 o 1.
- Número de grupo (`grupo`).

Este listado permite iterar sobre los vídeos que se analizarán en el lote actual, controlando así la ejecución por partes.

En este caso se configura el grupo inicial a analizar (`grupo_inicio = 62`) y se define cuántos grupos consecutivos se procesarán (`grupos = 9`).


In [ ]:
datos = os.path.join(folder_path, "videos_analisis.csv")
df = pd.read_csv(datos)

def extraer_lista_urls(df, grupo):
    # Filtrar por ese grupo
    df_filtrado = df[df["grupo"] == grupo].copy()

    # 📌 EXTRAER lista de tuplas (link, likes, views, grupo)
    lista_videos = list(zip(
        df_filtrado["link"],
        df_filtrado["categoria"],
        df_filtrado["grupo"]
    ))

    print(f"Primeras 5 tuplas del grupo {grupo}: {lista_videos[:5]}")
    return lista_videos

# =====================
# Elegir el grupo de videos a analizar
# =====================
grupo_inicio = 62
grupos = 9

## Procesamiento por lote y guardado de resultados

Este bloque ejecuta el procesamiento completo de los vídeos por grupos definidos previamente, automatizando todas las etapas del pipeline descritas en el capítulo 3 del TFM.

### `convert_json(obj)`

Función auxiliar para convertir objetos de tipo NumPy (enteros, flotantes, booleanos, arrays) a formatos compatibles con JSON estándar. Se utiliza durante el guardado de resultados para evitar errores de serialización.

---

### `procesar_lista_videos(urls)`

Procesa todos los vídeos de una lista `urls` (tuplas de enlace, categoría y grupo). Para cada vídeo:

1. Se descarga el vídeo (`yt-dlp`) y se extrae el audio (`ffmpeg`).
2. Se analiza el audio:
   - Se segmenta dinámicamente por pausas y cambios de énfasis.
   - Se transcribe el texto.
   - Se estima la emoción acústica por segmento.
3. Se analiza el vídeo:
   - Se extrae el fotograma central de cada segmento.
   - Se aplican modelos ligeros para detectar rostro, gestos, postura y manos.
4. Se integran los resultados (audio + vídeo + texto) en un diccionario estructurado.
5. Se limpia cualquier archivo temporal generado (audio y vídeo).

Se lleva registro del número de vídeos procesados, del tiempo total de ejecución y del número de vídeos exitosamente analizados.



En este bloque se itera por grupos consecutivos de vídeos (desde `grupo_inicio` hasta `grupo_inicio + grupos`), procesando todos los vídeos de cada grupo mediante `procesar_lista_videos()`.

Los resultados se guardan individualmente en un archivo JSON por grupo, en la ruta de salida (`json_path`). Cada archivo tiene el nombre:



In [ ]:
def convert_json(obj):
    if isinstance(obj, (np.integer, np.int_, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return str(obj)

def procesar_lista_videos(urls):
    resultados = {}
    tiempo_total=0
    n_videos_procesados=0
    n_videos_analizados=0
    for url, categoria, grupo in urls:
        n_videos_procesados+=1
        print(f"\n🔄 Procesando video número {n_videos_procesados}: {url}.")
        inicio = time.time()

        video_path = None
        audio_path = None

        try:
            video_path, video_id = download_video(url)
            audio_path = f"{video_id}.wav"
            extract_audio_from_video(video_path, audio_path)
            print("Procesando audio")

            resultado_audio = analizar_audio(audio_path)
            print("Procesando video")

            segmentos = procesar_video(video_path,resultado_audio["segmentos"])

            resultados[video_id] = {
                "video_id": video_id,
                "link": url,
                "tipo_comunicador": categoria,
                "grupo": grupo,
                "duracion_video": resultado_audio["duracion_video"],
                "texto_completo": resultado_audio["texto_completo"],
                "idioma": resultado_audio["idioma"],
                "segmentos": segmentos
            }

            n_videos_analizados+=1

        except Exception as e:
            print(f"❌ Error procesando {url}: {e}")
            continue  # Pasar al siguiente video

        finally:
            # Limpiar archivos temporales
            if video_path and os.path.exists(video_path):
                os.remove(video_path)
            if audio_path and os.path.exists(audio_path):
                os.remove(audio_path)
            clean_up()



        fin = time.time()
        duracion = round(fin - inicio, 2)

        tiempo_total+=duracion
        print(f"✅ Video {video_id} procesado en {duracion} segundos.")
        print(f"✅ {n_videos_analizados} videos analizados correctamente de {n_videos_procesados} videos procesados.")

        time.sleep(random.randint(3, 7))
    print(f"\n✅ Videos grupo {grupo} procesados en {tiempo_total/60:.2f} minutos.")
    return resultados


if __name__ == "__main__":

    for grupo in range(grupo_inicio, grupo_inicio+grupos):
        lista_videos = extraer_lista_urls(df, grupo)
        lista_urls = lista_videos
        print(f"✅ Procesando grupo {grupo}.")
        resultados_finales = procesar_lista_videos(lista_urls)

        ruta_salida = os.path.join(json_path, f"resultados_grupo_{grupo}.json")

        with open(ruta_salida, "w", encoding="utf-8") as f:
            json.dump(resultados_finales, f, indent=2, ensure_ascii=False, default=convert_json)

        print(f"✅ Datos guardados en 'resultados_grupo_{grupo}.json'.")

    print("Limpiando memoria y reiniciando runtime para liberar GPU...")
    os._exit(0)

Primeras 5 tuplas del grupo 1: [('https://ted.com/talks/sir_ken_robinson_do_schools_kill_creativity', 1, 1), ('https://ted.com/talks/annie_bosler_and_don_greene_how_to_practice_effectively_for_just_about_anything', 1, 1), ('https://ted.com/talks/andrew_solomon_how_the_worst_moments_in_our_lives_make_us_who_we_are', 1, 1), ('https://ted.com/talks/roselinde_torres_what_it_takes_to_be_a_great_leader', 1, 1), ('https://ted.com/talks/john_green_the_nerd_s_guide_to_learning_everything_online', 1, 1)]
✅ Procesando grupo 1.

🔄 Procesando video número 1: https://ted.com/talks/sir_ken_robinson_do_schools_kill_creativity.
Procesando audio
Procesando video
✅ Video 66 procesado en 125.59 segundos.
✅ 1 videos analizados correctamente de 1 videos procesados.

🔄 Procesando video número 2: https://ted.com/talks/annie_bosler_and_don_greene_how_to_practice_effectively_for_just_about_anything.
Procesando audio
Procesando video
✅ Video 24447 procesado en 102.71 segundos.
✅ 2 videos analizados correctamente

[download] Got error: HTTPSConnectionPool(host='pu.tedcdn.com', port=443): Read timed out. (read timeout=20.0)


Procesando audio
Procesando video
✅ Video 2305 procesado en 513.31 segundos.
✅ 5 videos analizados correctamente de 5 videos procesados.

🔄 Procesando video número 6: https://ted.com/talks/hamdi_ulukaya_the_anti_ceo_playbook.
Procesando audio
Procesando video
✅ Video 41225 procesado en 256.05 segundos.
✅ 6 videos analizados correctamente de 6 videos procesados.

🔄 Procesando video número 7: https://ted.com/talks/rutger_bregman_poverty_isn_t_a_lack_of_character_it_s_a_lack_of_cash.
Procesando audio
Procesando video
✅ Video 2785 procesado en 156.62 segundos.
✅ 7 videos analizados correctamente de 7 videos procesados.

🔄 Procesando video número 8: https://ted.com/talks/frank_warren_half_a_million_secrets.
Procesando audio
Procesando video
✅ Video 1416 procesado en 61.45 segundos.
✅ 8 videos analizados correctamente de 8 videos procesados.

🔄 Procesando video número 9: https://ted.com/talks/adam_savage_my_love_letter_to_cosplay.
Procesando audio
Procesando video
✅ Video 2552 procesado en 2

[download] Got error: HTTPSConnectionPool(host='pu.tedcdn.com', port=443): Read timed out. (read timeout=20.0)


Procesando audio
Procesando video
✅ Video 61300 procesado en 1399.5 segundos.
✅ 10 videos analizados correctamente de 10 videos procesados.

🔄 Procesando video número 11: https://ted.com/talks/adrienne_mayor_the_greek_myth_of_talos_the_first_robot.
Procesando audio
Procesando video
✅ Video 50986 procesado en 62.19 segundos.
✅ 11 videos analizados correctamente de 11 videos procesados.

🔄 Procesando video número 12: https://ted.com/talks/larry_lagerstrom_einstein_s_miracle_year.
Procesando audio
Procesando video
✅ Video 2754 procesado en 123.7 segundos.
✅ 12 videos analizados correctamente de 12 videos procesados.

🔄 Procesando video número 13: https://ted.com/talks/rives_if_i_controlled_the_internet.
Procesando audio
Procesando video
✅ Video 26 procesado en 50.79 segundos.
✅ 13 videos analizados correctamente de 13 videos procesados.

🔄 Procesando video número 14: https://ted.com/talks/enrico_ramirez_ruiz_your_body_was_forged_in_the_spectacular_death_of_stars.
Procesando audio
Procesan

...

❌ Error procesando https://ted.com/talks/sasha_dichter_the_generosity_experiment: ERROR: unable to download video data: HTTP Error 403: Forbidden

🔄 Procesando video número 22: https://ted.com/talks/susan_shaw_the_oil_spill_s_toxic_trade_off.
Procesando audio
Procesando video
✅ Video 925 procesado en 296.31 segundos.
✅ 21 videos analizados correctamente de 22 videos procesados.

🔄 Procesando video número 23: https://ted.com/talks/madhumita_murgia_how_data_brokers_sell_your_identity.


ERROR: unable to download video data: HTTP Error 403: Forbidden


❌ Error procesando https://ted.com/talks/madhumita_murgia_how_data_brokers_sell_your_identity: ERROR: unable to download video data: HTTP Error 403: Forbidden

🔄 Procesando video número 24: https://ted.com/talks/jasmine_cho_how_i_use_cookies_to_teach_history.


ERROR: unable to download video data: HTTP Error 403: Forbidden


❌ Error procesando https://ted.com/talks/jasmine_cho_how_i_use_cookies_to_teach_history: ERROR: unable to download video data: HTTP Error 403: Forbidden

🔄 Procesando video número 25: https://ted.com/talks/bob_nease_how_to_trick_yourself_into_good_behavior.
Procesando audio
Procesando video
✅ Video 9467 procesado en 222.89 segundos.
✅ 22 videos analizados correctamente de 25 videos procesados.

🔄 Procesando video número 26: https://ted.com/talks/maeve_higgins_why_the_good_immigrant_is_a_bad_narrative.
Procesando audio
Procesando video
✅ Video 41916 procesado en 200.62 segundos.
✅ 23 videos analizados correctamente de 26 videos procesados.

🔄 Procesando video número 27: https://ted.com/talks/jared_hill_how_i_leapt_from_a_responsible_no_to_an_impassioned_yes.
Procesando audio
Procesando video
✅ Video 13015 procesado en 178.52 segundos.
✅ 24 videos analizados correctamente de 27 videos procesados.

🔄 Procesando video número 28: https://ted.com/talks/ise_lyfe_we_are_not_mud.
Procesando aud